In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import statsmodels.api as sm
%matplotlib inline

C:\Python64\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df = pd.read_csv("../data/Bone Mineral Density.txt", sep='\t')
df.head()

,idnum,age,gender,spnbmd
0,1,11.70,male,0.018081
1,1,12.70,male,0.060109
2,1,13.75,male,0.005858
3,2,13.25,male,0.010264
4,2,14.30,male,0.210526


In [3]:
x_male = df[df.gender == 'male'].age.values
y_male = df[df.gender == 'male'].spnbmd.values
x_female = df[df.gender == 'female'].age.values
y_female = df[df.gender == 'female'].spnbmd.values
x_min, x_max = df.age.min(), df.age.max()

In [4]:
def dk(x, xi_k, xi_K):
    return (max(0, x-xi_k)**3) / (xi_K-xi_k)

def do_nc_expansion(x, xis):
    X = np.zeros(shape=(x.shape[0], len(xis)))
    for i in range(X.shape[0]):
        X[i, 0] = 1
        X[i, 1] = x[i]
        for k in range(2, len(xis)):
            X[i, k] = dk(x[i], xis[k-2], xis[-1]) - dk(x[i], xis[-2], xis[-1])
    return X

In [5]:
x_unique_male = np.unique(x_male)
N_male = do_nc_expansion(x_male, x_unique_male)
x_unique_female = np.unique(x_female)
N_female = do_nc_expansion(x_female, x_unique_female)

In [6]:
ls_male = sm.OLS(y_male, N_male).fit_regularized(alpha=0.00022, L1_wt=0)
ls_female = sm.OLS(y_female, N_female).fit_regularized(alpha=0.00022, L1_wt=0)

In [9]:
plt.figure(figsize=(7.5,5.7))
x = np.linspace(x_min, x_max, 100)
y_male_hat = ls_male.predict(do_nc_expansion(x, x_unique_male))
y_female_hat = ls_female.predict(do_nc_expansion(x, x_unique_female))
#plt.scatter(x_male, y_male, s=0.5, color='blue')
#plt.plot(x, y_male_hat, color='blue')
#plt.scatter(x_female, y_female, s=0.5, color='red')
#plt.plot(x, y_female_hat, color='red')

In [8]:
O = np.zeros(shape=(N_male.shape[1], N_male.shape[1]))

In [11]:
for i in range(O.shape[0]):
    for j in range(O.shape[1]):
        if i <= 1 or j <= 1:
            continue
        for k in range(len(x_unique_male)-1):
            multiplier = 

(152, 152)